# Using transformers for SeriesBasedCesnetDataset

### Import

In [1]:
import numpy as np
import logging

from cesnet_tszoo.utils.enums import AgreggationType, SourceType, TransformerType, DatasetType
from cesnet_tszoo.datasets import CESNET_TimeSeries24
from cesnet_tszoo.configs import SeriesBasedConfig # Series based dataset MUST use SeriesBasedConfig

from cesnet_tszoo.utils.transformer import Transformer # For creating custom Transformer

### Setting logger

In [2]:
logging.basicConfig(
    level=logging.INFO,
    format="[%(asctime)s][%(name)s][%(levelname)s] - %(message)s")

### Preparing dataset

In [3]:
series_based_dataset = CESNET_TimeSeries24.get_dataset(data_root="/some_directory/", source_type=SourceType.IP_ADDRESSES_SAMPLE, aggregation=AgreggationType.AGG_10_MINUTES, dataset_type=DatasetType.SERIES_BASED, display_details=True)

[2025-09-15 11:34:26,073][wrapper_dataset][INFO] - Dataset is series-based. Use cesnet_tszoo.configs.SeriesBasedConfig



Dataset details:

    AgreggationType.AGG_10_MINUTES
        Time indices: range(0, 40297)
        Datetime: (datetime.datetime(2023, 10, 9, 0, 3, 49, tzinfo=datetime.timezone.utc), datetime.datetime(2024, 7, 14, 21, 50, 52, tzinfo=datetime.timezone.utc))

    SourceType.IP_ADDRESSES_SAMPLE
        Time series indices: [ 11  20 101 103 118 ... 2003134 2008461 2011839 2022235 2044888], Length=1000; use 'get_available_ts_indices' for full list
        Features with default values: {'n_flows': 0, 'n_packets': 0, 'n_bytes': 0, 'n_dest_ip': 0, 'n_dest_asn': 0, 'n_dest_ports': 0, 'tcp_udp_ratio_packets': 0.5, 'tcp_udp_ratio_bytes': 0.5, 'dir_ratio_packets': 0.5, 'dir_ratio_bytes': 0.5, 'avg_duration': 0, 'avg_ttl': 0}
        
        Additional data: ['ids_relationship', 'weekends_and_holidays']
        


### Transformers

- Transformers are implemented as class.
    - You can create your own or use built-in one.
- Transformer is applied after `default_values` and fillers took care of missing values.
- One transformer is used for all time series.
- Transformer must implement `transform`.
- Transformer can implement `inverse_transform`.
- Transformer must implement `partial_fit` (unless transformer is already fitted and `partial_fit_initialized_transformers` is False).
- To use transformer, train set must be implemented (unless transformer is already fitted and `partial_fit_initialized_transformers` is False).
- You can change used transformer later with `update_dataset_config_and_initialize` or `apply_transformer`.

#### Built-in

In [4]:
# Options

## Supported
TransformerType.STANDARD_SCALER
TransformerType.L2_NORMALIZER
TransformerType.LOG_TRANSFORMER
TransformerType.MAX_ABS_SCALER
TransformerType.MIN_MAX_SCALER

<TransformerType.MIN_MAX_SCALER: 'min_max_scaler'>

In [5]:
config = SeriesBasedConfig(time_period=0.5, train_ts=500, features_to_take=["n_flows", "n_packets"],
                           transform_with=TransformerType.MIN_MAX_SCALER, nan_threshold=0.5, random_state=1500)
series_based_dataset.set_dataset_config_and_initialize(config, display_config_details=True, workers=0)

[2025-09-15 11:34:26,084][series_config][INFO] - Quick validation succeeded.
[2025-09-15 11:34:26,117][series_config][INFO] - Finalization and validation completed successfully.
[2025-09-15 11:34:26,122][cesnet_dataset][INFO] - Updating config on train/val/test/all and selected time period.
100%|██████████| 500/500 [00:00<00:00, 975.50it/s]
[2025-09-15 11:34:26,646][cesnet_dataset][INFO] - Dataset initialization complete. Configuration updated.
[2025-09-15 11:34:26,646][cesnet_dataset][INFO] - Config initialized successfully.



Config Details:
    Used for database: CESNET-TimeSeries24
    Aggregation: AgreggationType.AGG_10_MINUTES
    Source: SourceType.IP_ADDRESSES_SAMPLE

    Time series
        Train time series IDS: [182151  10158  65072  10196 338309 ... 175742 659213  11188  73422 483796], Length=60
        Val time series IDS: None
        Test time series IDS None
        All time series IDS [182151  10158  65072  10196 338309 ... 175742 659213  11188  73422 483796], Length=60
    Time periods
        Time period: range(0, 20149)
    Features
        Taken features: ['n_flows', 'n_packets']
        Default values: [0. 0.]
        Time series ID included: True
        Time included: True    
        Time format: TimeFormat.ID_TIME
    Fillers         
        Filler type: no_filler
    Transformers
        Transformer type: min_max_scaler
        Are transformers premade: False
        Are premade transformers partial_fitted: False
    Anomaly handler
        Anomaly handler type (train set): no_ano

In [6]:
series_based_dataset.get_train_df(workers=0).head(10)

,id_ip,id_time,n_flows,n_packets
0,182151.0,0.0,0.000079,2.240487e-07
1,182151.0,1.0,0.000045,1.280278e-07
2,182151.0,2.0,0.000045,1.280278e-07
3,182151.0,3.0,0.000056,1.600348e-07
4,182151.0,4.0,0.000090,2.560557e-07
5,182151.0,5.0,0.000034,9.602087e-08
6,182151.0,6.0,0.000068,1.920417e-07
7,182151.0,7.0,0.000045,1.280278e-07
8,182151.0,8.0,0.000102,3.840835e-07
9,182151.0,9.0,0.000000,0.000000e+00


In [7]:
series_based_dataset.get_transformers()

Or later with:

In [8]:
series_based_dataset.update_dataset_config_and_initialize(transform_with=TransformerType.MIN_MAX_SCALER, partial_fit_initialized_transformers="config", workers=0)
# Or
series_based_dataset.apply_transformer(transform_with=TransformerType.MIN_MAX_SCALER, partial_fit_initialized_transformers="config", workers=0)

[2025-09-15 11:34:26,842][cesnet_dataset][INFO] - Re-initialization is required.
[2025-09-15 11:34:26,872][series_config][INFO] - Finalization and validation completed successfully.
[2025-09-15 11:34:26,875][cesnet_dataset][INFO] - Updating config on train/val/test/all and selected time period.
100%|██████████| 60/60 [00:00<00:00, 402.02it/s]
[2025-09-15 11:34:27,027][cesnet_dataset][INFO] - Dataset initialization complete. Configuration updated.
[2025-09-15 11:34:27,027][cesnet_dataset][INFO] - Config initialized successfully.
[2025-09-15 11:34:27,027][cesnet_dataset][INFO] - Configuration has been changed successfuly.
[2025-09-15 11:34:27,029][cesnet_dataset][INFO] - Re-initialization is required.
[2025-09-15 11:34:27,058][series_config][INFO] - Finalization and validation completed successfully.
[2025-09-15 11:34:27,061][cesnet_dataset][INFO] - Updating config on train/val/test/all and selected time period.
100%|██████████| 60/60 [00:00<00:00, 367.75it/s]
[2025-09-15 11:34:27,226][c

#### Custom

You can create your own custom transformer. It is recommended to derive from Transformer base class.

In [9]:
class CustomTransformer(Transformer):
    def __init__(self):
        super().__init__()
        
        self.max = None
        self.min = None
    
    def transform(self, data):
        return (data - self.min) / (self.max - self.min)
    
    def fit(self, data):
        self.partial_fit(data)
    
    def partial_fit(self, data):
        
        if self.max is None and self.min is None:
            self.max = np.max(data, axis=0)
            self.min = np.min(data, axis=0)
            return
        
        temp_max = np.max(data, axis=0)
        temp = np.vstack((self.max, temp_max)) 
        self.max = np.max(temp, axis=0)
        
        temp_min = np.min(data, axis=0)
        temp = np.vstack((self.min, temp_min)) 
        self.min = np.min(temp, axis=0)     
        
    def inverse_transform(self, transformed_data):
        return transformed_data * (self.max - self.min) + self.min            

In [10]:
config = SeriesBasedConfig(time_period=0.5, train_ts=500, features_to_take=["n_flows", "n_packets"],
                           transform_with=CustomTransformer, nan_threshold=0.5, random_state=1500)
series_based_dataset.set_dataset_config_and_initialize(config, display_config_details=True, workers=0)

[2025-09-15 11:34:27,238][series_config][INFO] - Quick validation succeeded.
[2025-09-15 11:34:27,270][series_config][INFO] - Finalization and validation completed successfully.
[2025-09-15 11:34:27,274][cesnet_dataset][INFO] - Updating config on train/val/test/all and selected time period.
100%|██████████| 500/500 [00:00<00:00, 1069.49it/s]
[2025-09-15 11:34:27,744][cesnet_dataset][INFO] - Dataset initialization complete. Configuration updated.
[2025-09-15 11:34:27,744][cesnet_dataset][INFO] - Config initialized successfully.



Config Details:
    Used for database: CESNET-TimeSeries24
    Aggregation: AgreggationType.AGG_10_MINUTES
    Source: SourceType.IP_ADDRESSES_SAMPLE

    Time series
        Train time series IDS: [182151  10158  65072  10196 338309 ... 175742 659213  11188  73422 483796], Length=60
        Val time series IDS: None
        Test time series IDS None
        All time series IDS [182151  10158  65072  10196 338309 ... 175742 659213  11188  73422 483796], Length=60
    Time periods
        Time period: range(0, 20149)
    Features
        Taken features: ['n_flows', 'n_packets']
        Default values: [0. 0.]
        Time series ID included: True
        Time included: True    
        Time format: TimeFormat.ID_TIME
    Fillers         
        Filler type: no_filler
    Transformers
        Transformer type: CustomTransformer (Custom)
        Are transformers premade: False
        Are premade transformers partial_fitted: False
    Anomaly handler
        Anomaly handler type (train 

In [11]:
series_based_dataset.get_train_df(workers=0).head(10)

,id_ip,id_time,n_flows,n_packets
0,182151.0,0.0,0.000079,2.240487e-07
1,182151.0,1.0,0.000045,1.280278e-07
2,182151.0,2.0,0.000045,1.280278e-07
3,182151.0,3.0,0.000056,1.600348e-07
4,182151.0,4.0,0.000090,2.560557e-07
5,182151.0,5.0,0.000034,9.602087e-08
6,182151.0,6.0,0.000068,1.920417e-07
7,182151.0,7.0,0.000045,1.280278e-07
8,182151.0,8.0,0.000102,3.840835e-07
9,182151.0,9.0,0.000000,0.000000e+00


In [12]:
series_based_dataset.get_transformers()

Or later with:

In [13]:
series_based_dataset.update_dataset_config_and_initialize(transform_with=CustomTransformer, partial_fit_initialized_transformers="config", workers=0)
# Or
series_based_dataset.apply_transformer(transform_with=CustomTransformer, partial_fit_initialized_transformers="config", workers=0)

[2025-09-15 11:34:27,925][cesnet_dataset][INFO] - Re-initialization is required.
[2025-09-15 11:34:27,955][series_config][INFO] - Finalization and validation completed successfully.
[2025-09-15 11:34:27,958][cesnet_dataset][INFO] - Updating config on train/val/test/all and selected time period.
100%|██████████| 60/60 [00:00<00:00, 428.91it/s]
[2025-09-15 11:34:28,101][cesnet_dataset][INFO] - Dataset initialization complete. Configuration updated.
[2025-09-15 11:34:28,102][cesnet_dataset][INFO] - Config initialized successfully.
[2025-09-15 11:34:28,102][cesnet_dataset][INFO] - Configuration has been changed successfuly.
[2025-09-15 11:34:28,104][cesnet_dataset][INFO] - Re-initialization is required.
[2025-09-15 11:34:28,133][series_config][INFO] - Finalization and validation completed successfully.
[2025-09-15 11:34:28,137][cesnet_dataset][INFO] - Updating config on train/val/test/all and selected time period.
100%|██████████| 60/60 [00:00<00:00, 451.44it/s]
[2025-09-15 11:34:28,272][c

#### Using already fitted transformer

- When `partial_fit_initialized_transformer` is False (default value), transformer has no requirement for `partial_fit` nor for train set.

In [14]:
config = SeriesBasedConfig(time_period=0.5, train_ts=500, features_to_take=["n_flows", "n_packets"],
                           transform_with=CustomTransformer, nan_threshold=0.5, random_state=1500)
series_based_dataset.set_dataset_config_and_initialize(config, display_config_details=False, workers=0)

fitted_transformer = series_based_dataset.get_transformers()

[2025-09-15 11:34:28,277][series_config][INFO] - Quick validation succeeded.
[2025-09-15 11:34:28,310][series_config][INFO] - Finalization and validation completed successfully.
[2025-09-15 11:34:28,314][cesnet_dataset][INFO] - Updating config on train/val/test/all and selected time period.
100%|██████████| 500/500 [00:00<00:00, 1168.17it/s]
[2025-09-15 11:34:28,743][cesnet_dataset][INFO] - Dataset initialization complete. Configuration updated.
[2025-09-15 11:34:28,744][cesnet_dataset][INFO] - Config initialized successfully.


In [15]:
config = SeriesBasedConfig(time_period=0.5, train_ts=500, val_ts=500, features_to_take=["n_flows", "n_packets"],
                           transform_with=fitted_transformer, nan_threshold=0.5, random_state=999)
series_based_dataset.set_dataset_config_and_initialize(config, display_config_details=True, workers=0)

[2025-09-15 11:34:28,749][series_config][INFO] - Quick validation succeeded.
[2025-09-15 11:34:28,830][series_config][INFO] - Finalization and validation completed successfully.
[2025-09-15 11:34:28,835][cesnet_dataset][INFO] - Updating config on train/val/test/all and selected time period.
100%|██████████| 1000/1000 [00:00<00:00, 1154.41it/s]
[2025-09-15 11:34:29,703][cesnet_dataset][INFO] - Dataset initialization complete. Configuration updated.
[2025-09-15 11:34:29,704][cesnet_dataset][INFO] - Config initialized successfully.



Config Details:
    Used for database: CESNET-TimeSeries24
    Aggregation: AgreggationType.AGG_10_MINUTES
    Source: SourceType.IP_ADDRESSES_SAMPLE

    Time series
        Train time series IDS: [  4380  35210 322201    190 307400 ...  29194 617662 677144 211973 612051], Length=65
        Val time series IDS: [296195  43611  20342 100610  10703 ... 360850 121596 191587  92511 605036], Length=70
        Test time series IDS None
        All time series IDS [  4380  35210 322201    190 307400 ... 360850 121596 191587  92511 605036], Length=135
    Time periods
        Time period: range(0, 20149)
    Features
        Taken features: ['n_flows', 'n_packets']
        Default values: [0. 0.]
        Time series ID included: True
        Time included: True    
        Time format: TimeFormat.ID_TIME
    Fillers         
        Filler type: no_filler
    Transformers
        Transformer type: CustomTransformer (Custom)
        Are transformers premade: True
        Are premade transform

In [16]:
series_based_dataset.get_val_df(workers=0).head(10)

,id_ip,id_time,n_flows,n_packets
0,296195.0,0.0,0.000124,4.801044e-07
1,296195.0,1.0,0.000011,3.200696e-08
2,296195.0,2.0,0.000034,9.602087e-08
3,296195.0,3.0,0.000034,9.602087e-08
4,296195.0,4.0,0.000090,4.480974e-07
5,296195.0,5.0,0.000056,1.600348e-07
6,296195.0,6.0,0.000000,0.000000e+00
7,296195.0,7.0,0.000090,2.880626e-07
8,296195.0,8.0,0.000056,1.600348e-07
9,296195.0,9.0,0.000068,2.240487e-07


Below you can see how transformer works even without train set.

In [17]:
config = SeriesBasedConfig(time_period=0.5, val_ts=500, features_to_take=["n_flows", "n_packets"],
                           transform_with=fitted_transformer, nan_threshold=0.5, random_state=999)
series_based_dataset.set_dataset_config_and_initialize(config, display_config_details=True, workers=0)

[2025-09-15 11:34:29,918][series_config][INFO] - Quick validation succeeded.
[2025-09-15 11:34:29,950][series_config][INFO] - Finalization and validation completed successfully.
[2025-09-15 11:34:29,956][cesnet_dataset][INFO] - Updating config on train/val/test/all and selected time period.
100%|██████████| 500/500 [00:00<00:00, 1037.21it/s]
[2025-09-15 11:34:30,440][cesnet_dataset][INFO] - Dataset initialization complete. Configuration updated.
[2025-09-15 11:34:30,440][cesnet_dataset][INFO] - Config initialized successfully.



Config Details:
    Used for database: CESNET-TimeSeries24
    Aggregation: AgreggationType.AGG_10_MINUTES
    Source: SourceType.IP_ADDRESSES_SAMPLE

    Time series
        Train time series IDS: None
        Val time series IDS: [  4380  35210 322201    190 307400 ...  29194 617662 677144 211973 612051], Length=65
        Test time series IDS None
        All time series IDS [  4380  35210 322201    190 307400 ...  29194 617662 677144 211973 612051], Length=65
    Time periods
        Time period: range(0, 20149)
    Features
        Taken features: ['n_flows', 'n_packets']
        Default values: [0. 0.]
        Time series ID included: True
        Time included: True    
        Time format: TimeFormat.ID_TIME
    Fillers         
        Filler type: no_filler
    Transformers
        Transformer type: CustomTransformer (Custom)
        Are transformers premade: True
        Are premade transformers partial_fitted: False
    Anomaly handler
        Anomaly handler type (train s

In [18]:
series_based_dataset.get_val_df(workers=0).head(10)

,id_ip,id_time,n_flows,n_packets
0,4380.0,0.0,0.001253,0.000005
1,4380.0,1.0,0.001162,0.000004
2,4380.0,2.0,0.000756,0.000003
3,4380.0,3.0,0.000959,0.000004
4,4380.0,4.0,0.001219,0.000005
5,4380.0,5.0,0.001275,0.000005
6,4380.0,6.0,0.001456,0.000006
7,4380.0,7.0,0.001196,0.000004
8,4380.0,8.0,0.001433,0.000006
9,4380.0,9.0,0.001072,0.000004


##### Partial fitting on train set

Makes already fitted transformer to be fitted on new train set too. Must implement `partial_fit`.

In [19]:
config = SeriesBasedConfig(time_period=0.5, train_ts=500, val_ts=500, features_to_take=["n_flows", "n_packets"],
                           transform_with=fitted_transformer, partial_fit_initialized_transformer=True, nan_threshold=0.5, random_state=999)
series_based_dataset.set_dataset_config_and_initialize(config, display_config_details=True, workers=0)

[2025-09-15 11:34:30,614][series_config][INFO] - Quick validation succeeded.
[2025-09-15 11:34:30,646][series_config][INFO] - Finalization and validation completed successfully.
[2025-09-15 11:34:30,652][cesnet_dataset][INFO] - Updating config on train/val/test/all and selected time period.
100%|██████████| 1000/1000 [00:00<00:00, 1142.02it/s]
[2025-09-15 11:34:31,530][cesnet_dataset][INFO] - Dataset initialization complete. Configuration updated.
[2025-09-15 11:34:31,530][cesnet_dataset][INFO] - Config initialized successfully.



Config Details:
    Used for database: CESNET-TimeSeries24
    Aggregation: AgreggationType.AGG_10_MINUTES
    Source: SourceType.IP_ADDRESSES_SAMPLE

    Time series
        Train time series IDS: [  4380  35210 322201    190 307400 ...  29194 617662 677144 211973 612051], Length=65
        Val time series IDS: [756220 143746  65072  18683  78134 ...  100610 1490521      11  612150   78471], Length=70
        Test time series IDS None
        All time series IDS [  4380  35210 322201    190 307400 ...  100610 1490521      11  612150   78471], Length=135
    Time periods
        Time period: range(0, 20149)
    Features
        Taken features: ['n_flows', 'n_packets']
        Default values: [0. 0.]
        Time series ID included: True
        Time included: True    
        Time format: TimeFormat.ID_TIME
    Fillers         
        Filler type: no_filler
    Transformers
        Transformer type: CustomTransformer (Custom)
        Are transformers premade: True
        Are premade

In [20]:
series_based_dataset.get_val_df(workers=0).head(10)

,id_ip,id_time,n_flows,n_packets
0,756220.0,0.0,0.000090,0.000003
1,756220.0,1.0,0.000113,0.000004
2,756220.0,2.0,0.000068,0.000005
3,756220.0,3.0,0.000169,0.000010
4,756220.0,4.0,0.000158,0.000006
5,756220.0,5.0,0.000068,0.000006
6,756220.0,6.0,0.000090,0.000003
7,756220.0,7.0,0.000090,0.000002
8,756220.0,8.0,0.000158,0.000007
9,756220.0,9.0,0.000000,0.000000


#### Getting pre-transform value

- You can use `inverse_transform` for transformers you can get via `get_transformers()` to get pre-transform value.
- `inverse_transformer` expects input as numpy array of shape `(times, features)` where features do not contain ids.

In [21]:
config = SeriesBasedConfig(time_period=0.5, train_ts=500, features_to_take=["n_flows", "n_packets"],
                           transform_with=TransformerType.MIN_MAX_SCALER, nan_threshold=0.5, random_state=1500)
series_based_dataset.set_dataset_config_and_initialize(config, display_config_details=True, workers=0)

[2025-09-15 11:34:31,735][series_config][INFO] - Quick validation succeeded.
[2025-09-15 11:34:31,766][series_config][INFO] - Finalization and validation completed successfully.
[2025-09-15 11:34:31,771][cesnet_dataset][INFO] - Updating config on train/val/test/all and selected time period.
100%|██████████| 500/500 [00:00<00:00, 1052.31it/s]
[2025-09-15 11:34:32,248][cesnet_dataset][INFO] - Dataset initialization complete. Configuration updated.
[2025-09-15 11:34:32,248][cesnet_dataset][INFO] - Config initialized successfully.



Config Details:
    Used for database: CESNET-TimeSeries24
    Aggregation: AgreggationType.AGG_10_MINUTES
    Source: SourceType.IP_ADDRESSES_SAMPLE

    Time series
        Train time series IDS: [182151  10158  65072  10196 338309 ... 175742 659213  11188  73422 483796], Length=60
        Val time series IDS: None
        Test time series IDS None
        All time series IDS [182151  10158  65072  10196 338309 ... 175742 659213  11188  73422 483796], Length=60
    Time periods
        Time period: range(0, 20149)
    Features
        Taken features: ['n_flows', 'n_packets']
        Default values: [0. 0.]
        Time series ID included: True
        Time included: True    
        Time format: TimeFormat.ID_TIME
    Fillers         
        Filler type: no_filler
    Transformers
        Transformer type: min_max_scaler
        Are transformers premade: False
        Are premade transformers partial_fitted: False
    Anomaly handler
        Anomaly handler type (train set): no_ano

In [22]:
transformer = series_based_dataset.get_transformers()

data = None
for batch in series_based_dataset.get_train_dataloader():
    data = batch[0, :, 2:]
    break

transformer.inverse_transform(data)[:10]

[2025-09-15 11:34:32,254][cesnet_dataset][INFO] - Created new cached train_dataloader.


array([[ 7.,  7.],
       [ 4.,  4.],
       [ 4.,  4.],
       [ 5.,  5.],
       [ 8.,  8.],
       [ 3.,  3.],
       [ 6.,  6.],
       [ 4.,  4.],
       [ 9., 12.],
       [ 0.,  0.]])